In [1]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd
import ast
from scipy import stats

experiments = [
    {
        'name': 'random-uniform-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },

    {
        'name': 'random-pixel-fixed-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'fixed-pixel-variable-size',
        'filters': ['hsl', 'blur', 'merge'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)],
            'merge': [0.0, 0.14674845465465456489765465478976546548]
        }
    },
    
    {
        'name': 'hsl-force-first-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(0.0, 0.0, 0.0)]
        }
    },
    
    {
        'name': 'hsl-force-last-if',
        'filters': ['hsl'],
        'languages': ['c', 'asm1', 'asm2'],
        'params': {
            'hsl': [(99.0, 0.0, 0.0)]
        }
    }
]

command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

trim = 0.25

def runExperiment(filter, arguments):
    comm = command[filter].format(**arguments)
    delta = subprocess.check_output(comm, shell=True)
    delta = ast.literal_eval(delta.strip())
    rdelta = pd.Series(delta)
    
    return {
            'delta': delta,
            'mean': rdelta.mean(),
            'var': rdelta.var(),
            'tmean': stats.trim_mean(rdelta, trim),
            'min': rdelta.min()
        }

def experimentWorker(experiment, param, dataset, procImages):
    output = []

    for (f, l, src) in itertools.product(experiment['filters'], experiment['languages'], procImages):
        expOutput = {
            'experiment': experiment['name'],
            'make-param': param,
            'filename': os.path.splitext(os.path.basename(src))[0],
            'filter': f,
            'language': l
        }

        arguments = {
            'language': l,
            'filter': f,
            'src': src,
            'dst': '/dev/null'
        }
        
        if f == 'hsl':
            for (h, s, l) in experiment['params'][f]:
                pars = {
                        'h': h,
                        's': s,
                        'l': l
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args.update(pars)
                args.update(expOutput)
                output.append(args)
        elif f == 'merge':
            for alpha in experiment['params'][f]:
                pars = {
                        'alpha': alpha,
                        'src2': src
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args['alpha'] = alpha
                args.update(expOutput)
                output.append(args)
        else:
            args = runExperiment(f, arguments)
            args.update(expOutput)
            output.append(args)

    dataset += output
    del output

manager = multiprocessing.Manager()
dataset = manager.list([])

base_path = '/home/julian/Development/orga2-tp2'
src_path = os.path.join(base_path, 'src')
filters_path = os.path.join(src_path, 'filters')
test_path = os.path.join(src_path, 'tests')
makeparams = ['o0', 'o3', 'o3fast']

exp_img_path = os.path.join(base_path, 'exp', 'images')

for param in makeparams:
    subprocess.check_output('make -C "' + filters_path + '" ' + param, executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '"' , executable='/bin/bash', shell=True)
    
    jobs = []

    for experiment in experiments:
        images_path = os.path.join(exp_img_path, experiment['name'])

        procImages = []
        
        for img in os.listdir(images_path):
            rpath = os.path.join(images_path, img)

            if os.path.isfile(rpath):
                procImages.append(rpath)

        p = multiprocessing.Process(target=experimentWorker, args=(experiment, param, dataset, procImages))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()

In [3]:
ds = []

for x in dataset:
    ds.append(x)
    
ds = pd.DataFrame(ds)

In [5]:
ds.to_csv('dataset.csv')